In [1]:
%load_ext autoreload

%autoreload 1

In [13]:
import xarray as xr
from pathlib import Path
import numpy as np
import numexpr as ne

import time
from analysis_utils import load_data_set, calc_temp_spac_means

%aimport analysis_utils

In [29]:
job_path = Path("/users/PAS0409/madelinescyphers/palm/current_version/JOBS/20220308T183847")
data_path = job_path / "OUTPUT/20220308T183847_3d.nc"
lad_path = job_path / "INPUT/20220308T183847_static"
ds = load_data_set(data_path)
ds_lad = xr.open_dataset(lad_path, chunks="auto")

/users/PAS0409/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [35]:
try:
    ds.chunks
except ValueError:
    ds = ds.unify_chunks()
try:
    ds_lad.chunks
except ValueError:
    ds_lad = ds_lad.unify_chunks()
     

In [31]:
ds_lad.chunks

Frozen({'zlad': (6,), 'y': (432,), 'x': (192,)})

In [36]:
ds.chunks

Frozen({'x': (64, 64, 64), 'y': (144, 144, 144), 'xu': (64, 64, 64), 'yv': (144, 144, 144), 'time': (24, 24, 24), 'zu_3d': (96, 97), 'zw_3d': (96, 97)})

In [37]:
dz = 3

z_scalar  = np.arange(150//dz, 300//dz)

urban_ratio = 0.5
y_domain = np.arange(int(ds.s.y.size * urban_ratio), ds.s.y.size)

In [5]:
# ds_lad.isel(zlad=1).lad.values

In [6]:
# ds

In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)
# print(time.time() - s)
# # ubar_total = np.sqrt(np.square(ds.u.values) + np.square(ds.v.values) + np.square(ds.w.values))

In [5]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

23.01164722442627


In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u) + np.square(sub_ds.v) + np.square(sub_ds.w))
# # ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

In [41]:
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)

In [47]:
s = time.time()
sub_ds.u**2
print(time.time() - s)


dask.array<pow, shape=(72, 50, 216, 192), dtype=float32, chunksize=(24, 46, 144, 64), chunktype=numpy.ndarray>
0.0018227100372314453


In [ ]:
u2_2 = np.square(sub_ds.u)

In [58]:
%time u2.compute()

CPU times: user 10.3 s, sys: 27 s, total: 37.3 s
Wall time: 6.57 s


<xarray.DataArray 'u' (time: 72, zu_3d: 50, y: 216, xu: 192)>
array([[[[1.20187120e-03, 5.82910317e-04, 5.15852764e-04, ...,
          7.52276286e-09, 1.77709473e-04, 2.25754575e-05],
         [2.26918129e-08, 5.15691005e-04, 2.44591036e-03, ...,
          3.99560231e-04, 2.30569131e-04, 9.48354170e-07],
         [1.81410671e-03, 2.56682839e-03, 5.87040698e-03, ...,
          8.06077034e-04, 3.45382032e-05, 3.92679649e-05],
         ...,
         [5.80045511e-04, 2.13915337e-05, 9.17660000e-05, ...,
          1.92366529e-03, 1.84880849e-03, 4.38583374e-04],
         [7.68424070e-04, 2.60249805e-03, 2.35586049e-04, ...,
          1.08057319e-03, 2.12310508e-04, 9.75933654e-05],
         [2.22572125e-03, 2.58437311e-03, 5.14582149e-04, ...,
          5.32266240e-05, 8.35219225e-07, 4.84862685e-05]],

        [[1.13997507e-04, 3.34387482e-03, 3.35798359e-05, ...,
          9.64320824e-03, 3.21440655e-03, 1.30557048e-04],
         [4.80260613e-04, 3.38399317e-03, 1.92172464e-03, ...,
          6.77351188e-03, 3.63878487e-03, 2.10809149e-03],
         [2.98883038e-04, 4.52923588e-03, 8.46781023e-03, ...,
          2.79667112e-03, 1.22261979e-03, 1.10638421e-03],
...
          1.33965969e-07, 1.17135912e-07, 1.01032178e-07],
         [8.39452241e-08, 6.95892126e-08, 5.63059821e-08, ...,
          1.32427274e-07, 1.15471749e-07, 9.92742244e-08],
         [8.21748500e-08, 6.78160603e-08, 5.45703749e-08, ...,
          1.30878149e-07, 1.13817222e-07, 9.75445218e-08]],

        [[3.85853241e-07, 3.69492341e-07, 3.52311588e-07, ...,
          4.29273825e-07, 4.15809467e-07, 4.01315447e-07],
         [3.78335955e-07, 3.62651690e-07, 3.46150870e-07, ...,
          4.19762387e-07, 4.06955593e-07, 3.93126584e-07],
         [3.70271863e-07, 3.55286772e-07, 3.39489105e-07, ...,
          4.09642809e-07, 3.97512736e-07, 3.84369343e-07],
         ...,
         [7.91709027e-08, 6.60255921e-08, 5.38107834e-08, ...,
          1.23255532e-07, 1.07883110e-07, 9.31550659e-08],
         [7.75497639e-08, 6.44005453e-08, 5.22169756e-08, ...,
          1.21831846e-07, 1.06365896e-07, 9.15708469e-08],
         [7.59698153e-08, 6.28294572e-08, 5.06878379e-08, ...,
          1.20400955e-07, 1.04859083e-07, 9.00132093e-08]]]],
      dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
Dimensions without coordinates: zu_3d

In [60]:
%timeit u2_2.compute()

6.73 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

20.63394570350647


In [39]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

22.61529040336609


In [5]:
ubar_scalar = ubar_scalar.mean(skipna=True)

<xarray.DataArray 'u' (time: 72, zu_3d: 193, y: 432, xu: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 568.5 571.5 574.5 577.5
Dimensions without coordinates: zu_3d
Attributes:
    units:      m/s
    long_name:  u

In [ ]:
scalar_gradient = ds.s.mean(skipna=True)

In [11]:
TempSpacMeanU, TempSpacMeanV, TempSpacMeanW = calc_temp_spac_means(ds)

In [ ]:
uw = (ds.w - TempSpacMeanW) * (ds.u - TempSpacMeanU);
vw = (ds.w - TempSpacMeanW) * (ds.v - TempSpacMeanV);

In [ ]:
np.power(np.nanmean(uw)**2 + np.nanmean(vw)**2, 1/4)

In [13]:
ustar = (squeeze(nanmean(nanmean(uw,1),2).^2 + nanmean(nanmean(vw,1),2).^2)).^0.25;

KeyError: 'values'

In [8]:
ds.w

<xarray.DataArray 'w' (time: 72, zw_3d: 193, y: 432, x: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zw_3d_   (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
Dimensions without coordinates: zw_3d
Attributes:
    units:      m/s
    long_name:  w

In [ ]:
ubar = ubar_total

In [23]:
type(ubar_total)

numpy.ndarray

In [7]:
ne.evaluate("exp(2)")

array(7.3890561)

In [21]:
scalar_grad = ds.s.isel(y=y_domain, zu_3d=z_scalar)

In [16]:
ds.s.y.size * urban_ratio

216.0

In [6]:
ds.s.isel(time=60, zu_3d=10).v

array([[0.1304376 , 0.1125954 , 0.09213795, ..., 0.0807151 , 0.14683247,
        0.14305805],
       [0.12404612, 0.1070879 , 0.07839098, ..., 0.08024671, 0.13751815,
        0.1399681 ],
       [0.12678802, 0.12038978, 0.10291763, ..., 0.08745276, 0.13599472,
        0.13938259],
       ...,
       [0.12819828, 0.14347515, 0.18452443, ..., 0.0778394 , 0.13670161,
        0.14400712],
       [0.13251767, 0.12401052, 0.14600988, ..., 0.07967169, 0.15714346,
        0.1510945 ],
       [0.13553588, 0.11933067, 0.11963697, ..., 0.082677  , 0.15785772,
        0.14794137]], dtype=float32)

In [38]:
da = xr.DataArray(np.arange(25).reshape(5, 5), dims=("x", "y"))
da

<xarray.DataArray (x: 5, y: 5)>
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])
Dimensions without coordinates: x, y

In [37]:
tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
da = da.isel(x=tgt_x, y=tgt_y)
da

ValueError: Dimensions {'y', 'x'} do not exist. Expected one or more of Frozen({'points': 5})

In [39]:
da.values

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [3]:
a = np.arange(25).reshape((5,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])